In [42]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
import copy

In [43]:
# Load only relevant columns, TV shows from USA
netflix_data = pd.read_csv('netflix_titles.csv', usecols=['type', 'director', 'country', 'rating', 'duration'])
tv_shows = netflix_data[(netflix_data['type'] == 'TV Show') & (netflix_data['country'] == 'United States')].drop('type', axis=1)

# Convert 'duration' to numeric values, handling "Seasons" separately
tv_shows['duration'] = tv_shows['duration'].apply(lambda x: int(x.split()[0]) if 'Season' in x else None)
tv_shows = tv_shows.dropna(subset=['duration'])

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Label encode the 'rating' column
tv_shows['rating'] = label_encoder.fit_transform(tv_shows['rating'])

# Label encode the 'director' column
tv_shows['director'] = label_encoder.fit_transform(tv_shows['director'])


In [44]:
def load_netflix(tv_shows):
    # fetch dataset
    features = [ 'rating', 'director']
    X = tv_shows[features]
    y = tv_shows['duration']

    # with this random seed, no null value is included in the test data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    X_train = copy.deepcopy(X_train).reset_index(drop=True)
    X_test = copy.deepcopy(X_test).reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)

    return X_train, X_test, y_train, y_test

# first impute the data and make it hypothetically clean
def load_netflix_cleaned(tv_shows):
    # fetch dataset
    features = [ 'rating', 'director']
    X = tv_shows[features]
    y = tv_shows['duration']

    # assumed gt imputation
    imputer = KNNImputer(n_neighbors=10)
    X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

    # Split the data into training and testing sets with a random seed
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    X_train = copy.deepcopy(X_train).reset_index(drop=True)
    X_test = copy.deepcopy(X_test).reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)

    return X_train, X_test, y_train, y_test